<a href="https://colab.research.google.com/github/dagmaros27/AIMS_Notebooks/blob/main/CUDA_Practical_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CUDA Programming on NVIDIA GPUs**

# **Practical 3**

Again make sure the correct Runtime is being used, by clicking on the Runtime option at the top, then "Change runtime type", and selecting an appropriate GPU such as the T4.

Then verify with the instruction below the details of the GPU which is available to you.  

In [1]:
!nvidia-smi


Wed Jan 21 15:53:10 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---

First we upload two header files from the course webpage.

In [2]:
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h


--2026-01-21 15:53:11--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:201::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27832 (27K) [text/x-chdr]
Saving to: ‘helper_cuda.h’

helper_cuda.h       100%[===================>]  27.18K  --.-KB/s    in 0.1s    

2026-01-21 15:53:12 (208 KB/s) - ‘helper_cuda.h’ saved [27832/27832]

--2026-01-21 15:53:12--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:201::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14875 (15K) [text/x-chdr]
Saving to: ‘helper_string.h’

helper_string.h     100%[===================>]  14.53K 



---

The next step is to create the file laplace3d.cu which includes within it a reference C++ routine against which the CUDA results are compared.

In [1]:
%%writefile laplace3d.cu

////////////////////////////////////////////////////////////////////////
//
// Program to solve Laplace equation on a regular 3D grid
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
                              const float* __restrict__ d_u1,
                                    float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  indg = i + j*NX;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  if ( i>=0 && i<=NX-1 && j>=0 && j<=NY-1 ) {

    for (k=0; k<NZ; k++) {

      if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) {
        u2 = d_u1[indg];  // Dirichlet b.c.'s
      }
      else {
        u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
             + d_u1[indg-JOFF] + d_u1[indg+JOFF]
             + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
      }
      d_u2[indg] = u2;

      indg += KOFF;
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}

////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=1024, NY=1024, NZ=1024, REPEAT=20,
            BLOCK_X, BLOCK_Y, bx, by, i, j, k;
  float    *h_u1, *h_u2, *h_foo,
           *d_u1, *d_u2, *d_foo;

  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

  //cudaEventRecord(start);
  //for (i=0; i<REPEAT; i++) {
  //  Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
  //  h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
  //}

  //cudaEventRecord(stop);
  //cudaEventSynchronize(stop);
  //cudaEventElapsedTime(&milli, start, stop);
  //printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  BLOCK_X = 64; // number of threads
  BLOCK_Y = 16; // in each direction

  bx = 1 + (NX-1)/BLOCK_X; // number of blocks
  by = 1 + (NY-1)/BLOCK_Y; // in each direction

  dim3 dimGrid(bx,by);
  dim3 dimBlock(BLOCK_X,BLOCK_Y);

  // Execute GPU kernel

  cudaEventRecord(start);

  for (i=0; i<REPEAT; i++) {
    GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
    getLastCudaError("GPU_laplace3d execution failed\n");

    d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;   // swap d_u1 and d_u2
  }

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Block dimensions: %d x %d \n", BLOCK_X,BLOCK_Y);
  printf("%dx GPU_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);

  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}


Overwriting laplace3d.cu


In [24]:
%%writefile laplace3d.cu


////////////////////////////////////////////////////////////////////////
//
// Program to solve Laplace equation on a regular 3D grid
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
                              const float* __restrict__ d_u1,
                                    float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  indg = i + j*NX;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  if ( i>=0 && i<=NX-1 && j>=0 && j<=NY-1 ) {

    for (k=0; k<NZ; k++) {

      if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) { // bound elements
        u2 = d_u1[indg];  // Dirichlet b.c.'s
      }
      else {
        u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
             + d_u1[indg-JOFF] + d_u1[indg+JOFF]
             + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
      }
      d_u2[indg] = u2;

      indg += KOFF;
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}

////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=1024, NY=1024, NZ=1024, REPEAT=20,
          i, j, k;
  float    *h_u1, *h_u2,
         *d_u1, *d_u2, *d_foo;


  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

//   cudaEventRecord(start);
//   for (i=0; i<REPEAT; i++) {
//     Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
//     h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
//   }

//   cudaEventRecord(stop);
//   cudaEventSynchronize(stop);
//   cudaEventElapsedTime(&milli, start, stop);
//   printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  int block_sizes[] = {2, 4, 8, 16, 24, 32, 64};   // try common warp-friendly sizes
  int nSizes = sizeof(block_sizes)/sizeof(block_sizes[0]);

  float bestTime = 1e30;
  int bestBX = 0, bestBY = 0;

  for (int ix = 0; ix < nSizes; ix++) {
  for (int iy = 0; iy < nSizes; iy++) {

    int BLOCK_X = block_sizes[ix];
    int BLOCK_Y = block_sizes[iy];

    // Skip illegal configurations (>1024 threads per block)
    if (BLOCK_X * BLOCK_Y > 1024) continue;

    int bx = 1 + (NX-1)/BLOCK_X;
    int by = 1 + (NY-1)/BLOCK_Y;

    dim3 dimGrid(bx, by);
    dim3 dimBlock(BLOCK_X, BLOCK_Y);

    cudaDeviceSynchronize();

    cudaEventRecord(start);

    for (i = 0; i < REPEAT; i++) {
    GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
    getLastCudaError("GPU_laplace3d execution failed\n");

    d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;
    }

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milli, start, stop);

    printf("Block %2d x %2d  ->  %.3f ms\n", BLOCK_X, BLOCK_Y, milli);

    if (milli < bestTime) {
    bestTime = milli;
    bestBX = BLOCK_X;
    bestBY = BLOCK_Y;
    }
  }
  }

printf("\n==== BEST CONFIGURATION ====\n");
printf("Block: %d x %d   Time: %.3f ms\n", bestBX, bestBY, bestTime);


  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);

  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}



Overwriting laplace3d.cu



---

We can now compile and run the executable.


In [2]:
!nvcc laplace3d.cu -o laplace3d -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

laplace3d.cu(93): warning #177-D: variable "h_foo" was declared but never referenced
    float *h_u1, *h_u2, *h_foo,
                         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z13GPU_laplace3dxxxPKfPf' for 'sm_70'
ptxas info    : Function properties for _Z13GPU_laplace3dxxxPKfPf
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 64 registers, 392 bytes cmem[0]


In [3]:
!./laplace3d

Grid dimensions: 1024 x 1024 x 1024 

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 931.0 (ms) 

Block dimensions: 64 x 16 
20x GPU_laplace3d: 1201.7 (ms) 

Copy u2 to host: 2926.4 (ms) 

rms error = 0.066320 


```
Grid dimensions: 1024 x 1024 x 1024

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 938.8 (ms)

Block  2 x  2  ->  5071.906 ms
Block  2 x  4  ->  5656.572 ms
Block  2 x  8  ->  10098.975 ms
Block  2 x 16  ->  18532.633 ms
Block  2 x 24  ->  22519.248 ms
Block  2 x 32  ->  23792.822 ms
Block  2 x 64  ->  25030.516 ms
Block  4 x  2  ->  5273.936 ms
Block  4 x  4  ->  6782.370 ms
Block  4 x  8  ->  7705.290 ms
Block  4 x 16  ->  11683.622 ms
Block  4 x 24  ->  11806.559 ms
Block  4 x 32  ->  11969.221 ms
Block  4 x 64  ->  12230.748 ms
Block  8 x  2  ->  2203.408 ms
Block  8 x  4  ->  2161.137 ms
Block  8 x  8  ->  3006.282 ms
Block  8 x 16  ->  3431.023 ms
Block  8 x 24  ->  3469.136 ms
Block  8 x 32  ->  3829.409 ms
Block  8 x 64  ->  4184.480 ms
Block 16 x  2  ->  1661.684 ms
Block 16 x  4  ->  1852.686 ms
Block 16 x  8  ->  1842.868 ms
Block 16 x 16  ->  1930.509 ms
Block 16 x 24  ->  1839.395 ms
Block 16 x 32  ->  2090.338 ms
Block 16 x 64  ->  2002.745 ms
Block 24 x  2  ->  2131.153 ms
Block 24 x  4  ->  1809.894 ms
Block 24 x  8  ->  1870.719 ms
Block 24 x 16  ->  1694.615 ms
Block 24 x 24  ->  1728.848 ms
Block 24 x 32  ->  1756.730 ms
Block 32 x  2  ->  1465.997 ms
Block 32 x  4  ->  1280.623 ms
Block 32 x  8  ->  1237.430 ms
Block 32 x 16  ->  1243.163 ms
Block 32 x 24  ->  1272.825 ms
Block 32 x 32  ->  1270.032 ms
Block 64 x  2  ->  1483.745 ms
Block 64 x  4  ->  1325.004 ms
Block 64 x  8  ->  1231.041 ms
Block 64 x 16  ->  1191.235 ms

==== BEST CONFIGURATION ====
Block: 64 x 16   Time: 1191.235 ms
Copy u2 to host: 2908.7 (ms)

rms error = 0.125000




---
By going back to the previous code block you can modify the code to complete the initial Practical 3 exercises. Remember to first make your own copy of the notebook so that you are able to edit it.

For students doing this as an assignment to be assessed, you should again add your name to the title of the notebook (as in "Practical 3 -- Mike Giles.ipynb"), make it shared (see the Share option in the top-right corner) and provide the shared link as the submission mechanism.


---

For the later parts of Practical 3, the instructions below create, compile and execute a second version of the code in which each CUDA thread computes the value for just one 3D grid point.

In [ ]:
%%writefile laplace3d_new.cu

//
// Program to solve Laplace equation on a regular 3D grid
//

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
	         	      const float* __restrict__ d_u1,
			            float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  k    = threadIdx.z + blockIdx.z*blockDim.z;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  indg = i + j*JOFF + k*KOFF;

  if (i>=0 && i<=NX-1 && j>=0 && j<=NY-1 && k>=0 && k<=NZ-1) {
    if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) {
      u2 = d_u1[indg];  // Dirichlet b.c.'s
    }
    else {
      u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
           + d_u1[indg-JOFF] + d_u1[indg+JOFF]
           + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
    }
    d_u2[indg] = u2;
  }
}

////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=512, NY=512, NZ=512, REPEAT=20,
            BLOCK_X, BLOCK_Y, BLOCK_Z,bx, by, bz, i, j, k;
  float    *h_u1, *h_u2, *h_foo,
           *d_u1, *d_u2, *d_foo;

  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

  cudaEventRecord(start);
  for (i=0; i<REPEAT; i++) {
    Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
    h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
  }

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  BLOCK_X = 8; // number of threads
  BLOCK_Y = 8; // in each direction
  BLOCK_Z = 8; // of 3D thread block

  bx = 1 + (NX-1)/BLOCK_X; // number of blocks
  by = 1 + (NY-1)/BLOCK_Y; // in each direction
  bz = 1 + (NZ-1)/BLOCK_Z; // of 3D grid of blocks

  dim3 dimGrid(bx,by,bz);
  dim3 dimBlock(BLOCK_X,BLOCK_Y,BLOCK_Z);

  // Execute GPU kernel

  cudaEventRecord(start);

  for (i=0; i<REPEAT; i++) {
    GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
    getLastCudaError("GPU_laplace3d execution failed\n");

    d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;   // swap d_u1 and d_u2
  }

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Block dimensions: %d x %d x %d\n", BLOCK_X,BLOCK_Y,BLOCK_Z);
  printf("%dx GPU_laplace3d_new: %.1f (ms) \n\n", REPEAT, milli);

  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);

  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}


In [3]:
%%writefile laplace3d_new.cu


//
// Program to solve Laplace equation on a regular 3D grid
//

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
	         	      const float* __restrict__ d_u1,
			            float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  k    = threadIdx.z + blockIdx.z*blockDim.z;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  indg = i + j*JOFF + k*KOFF;

  if (i>=0 && i<=NX-1 && j>=0 && j<=NY-1 && k>=0 && k<=NZ-1) {
    if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) {
      u2 = d_u1[indg];  // Dirichlet b.c.'s
    }
    else {
      u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
           + d_u1[indg-JOFF] + d_u1[indg+JOFF]
           + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
    }
    d_u2[indg] = u2;
  }
}

////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=1024, NY=1024, NZ=1024, REPEAT=20,
            BLOCK_X, BLOCK_Y, BLOCK_Z,bx, by, bz, i, j, k;
  float    *h_u1, *h_u2,
           *d_u1, *d_u2, *d_foo;

  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

//   cudaEventRecord(start);
//   for (i=0; i<REPEAT; i++) {
//     Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
//     h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
//   }

//   cudaEventRecord(stop);
//   cudaEventSynchronize(stop);
//   cudaEventElapsedTime(&milli, start, stop);
//   printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  int block_sizes[] = {2, 4, 8, 16, 24, 32, 64};
  int nSizes = sizeof(block_sizes)/sizeof(block_sizes[0]);

  float bestTime = 1e30;
  int bestBX = 0, bestBY = 0, bestBZ = 0;

  for (int ix = 0; ix < nSizes; ix++) {
    for (int iy = 0; iy < nSizes; iy++) {
      for (int iz = 0; iz < nSizes; iz++) {

        BLOCK_X = block_sizes[ix];
        BLOCK_Y = block_sizes[iy];
        BLOCK_Z = block_sizes[iz];

        if (BLOCK_X * BLOCK_Y * BLOCK_Z > 1024) continue;

        bx = 1 + (NX-1)/BLOCK_X;
        by = 1 + (NY-1)/BLOCK_Y;
        bz = 1 + (NZ-1)/BLOCK_Z;

        dim3 dimGrid(bx, by, bz);
        dim3 dimBlock(BLOCK_X, BLOCK_Y, BLOCK_Z);

        cudaDeviceSynchronize();
        cudaEventRecord(start);

        for (i=0; i<REPEAT; i++) {
          GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
          getLastCudaError("GPU_laplace3d execution failed\n");

          d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;
        }

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&milli, start, stop);

        printf("Block %d x %d x %d  ->  %.3f ms\n", BLOCK_X, BLOCK_Y, BLOCK_Z, milli);

        if (milli < bestTime) {
          bestTime = milli;
          bestBX = BLOCK_X; bestBY = BLOCK_Y; bestBZ = BLOCK_Z;
        }
      }
    }
  }

  printf("\n==== BEST CONFIGURATION ====\n");
  printf("Block: %d x %d x %d   Time: %.3f ms\n", bestBX, bestBY, bestBZ, bestTime);

  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);

  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}


Writing laplace3d_new.cu


In [4]:
!nvcc laplace3d_new.cu -o laplace3d_new -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z13GPU_laplace3dxxxPKfPf' for 'sm_70'
ptxas info    : Function properties for _Z13GPU_laplace3dxxxPKfPf
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 21 registers, 392 bytes cmem[0]


In [5]:
!./laplace3d_new

Grid dimensions: 1024 x 1024 x 1024 

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 975.4 (ms) 

Block 2 x 2 x 2  ->  4028.775 ms
Block 2 x 2 x 4  ->  3152.670 ms
Block 2 x 2 x 8  ->  3131.752 ms
Block 2 x 2 x 16  ->  3210.297 ms
Block 2 x 2 x 24  ->  3272.550 ms
Block 2 x 2 x 32  ->  3401.198 ms
Block 2 x 2 x 64  ->  3515.413 ms
Block 2 x 4 x 2  ->  3418.371 ms
Block 2 x 4 x 4  ->  3346.674 ms
Block 2 x 4 x 8  ->  3358.173 ms
Block 2 x 4 x 16  ->  3472.517 ms
Block 2 x 4 x 24  ->  3516.457 ms
Block 2 x 4 x 32  ->  3552.722 ms
Block 2 x 4 x 64  ->  3617.280 ms
Block 2 x 8 x 2  ->  3561.329 ms
Block 2 x 8 x 4  ->  3499.206 ms
Block 2 x 8 x 8  ->  3572.329 ms
Block 2 x 8 x 16  ->  3598.354 ms
Block 2 x 8 x 24  ->  3623.103 ms
Block 2 x 8 x 32  ->  3678.904 ms
Block 2 x 8 x 64  ->  3999.664 ms
Block 2 x 16 x 2  ->  3843.965 ms
Block 2 x 16 x 4  ->  3857.678 ms
Block 2 x 16 x 8  ->  3827.151 ms
Block 2 x 16 x 16  ->  3827.921 ms
Block 2 x 16 x 24  ->  3905.278 ms
B

```
Grid dimensions: 1024 x 1024 x 1024

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 943.5 (ms)

Block  4 x  4 x  4  ->  1756.328 ms
Block  4 x  4 x  8  ->  1732.503 ms
Block  4 x  4 x 16  ->  1758.043 ms
Block  4 x  4 x 24  ->  1841.743 ms
Block  4 x  4 x 32  ->  1834.283 ms
Block  4 x  4 x 64  ->  2028.740 ms
Block  4 x  8 x  4  ->  1810.498 ms
Block  4 x  8 x  8  ->  1802.894 ms
Block  4 x  8 x 16  ->  1854.673 ms
Block  4 x  8 x 24  ->  2113.159 ms
Block  4 x  8 x 32  ->  2006.687 ms
Block  4 x 16 x  4  ->  1864.720 ms
Block  4 x 16 x  8  ->  1896.886 ms
Block  4 x 16 x 16  ->  1999.147 ms
Block  4 x 24 x  4  ->  1944.521 ms
Block  4 x 24 x  8  ->  2138.433 ms
Block  4 x 32 x  4  ->  1970.973 ms
Block  4 x 32 x  8  ->  2055.291 ms
Block  4 x 64 x  4  ->  2111.740 ms
Block  8 x  4 x  4  ->  1108.211 ms
Block  8 x  4 x  8  ->  1121.353 ms
Block  8 x  4 x 16  ->  1186.890 ms
Block  8 x  4 x 24  ->  1515.791 ms
Block  8 x  4 x 32  ->  1438.392 ms
Block  8 x  8 x  4  ->  1145.758 ms
Block  8 x  8 x  8  ->  1188.332 ms
Block  8 x  8 x 16  ->  1410.370 ms
Block  8 x 16 x  4  ->  1223.725 ms
Block  8 x 16 x  8  ->  1419.672 ms
Block  8 x 24 x  4  ->  1553.836 ms
Block  8 x 32 x  4  ->  1454.891 ms
Block 16 x  4 x  4  ->  965.029 ms
Block 16 x  4 x  8  ->  993.520 ms
Block 16 x  4 x 16  ->  1168.931 ms
Block 16 x  8 x  4  ->  1050.348 ms
Block 16 x  8 x  8  ->  1176.040 ms
Block 16 x 16 x  4  ->  1212.088 ms
Block 24 x  4 x  4  ->  1116.898 ms
Block 24 x  4 x  8  ->  1249.454 ms
Block 24 x  8 x  4  ->  1277.215 ms
Block 32 x  4 x  4  ->  954.941 ms
Block 32 x  4 x  8  ->  1039.336 ms
Block 32 x  8 x  4  ->  1086.807 ms
Block 64 x  4 x  4  ->  1036.007 ms

==== BEST CONFIGURATION ====
Block: 32 x 4 x 4  Time: 954.941 ms
Copy u2 to host: 2955.0 (ms)

rms error = 0.125000



---

The next instructions check how many fp32 and integer instructions are performed by the two versions

In [ ]:
!ncu --metrics "smsp__sass_thread_inst_executed_op_fp32_pred_on.sum,smsp__sass_thread_inst_executed_op_integer_pred_on.sum" ./laplace3d
!ncu --metrics "smsp__sass_thread_inst_executed_op_fp32_pred_on.sum,smsp__sass_thread_inst_executed_op_integer_pred_on.sum" ./laplace3d_new

Grid dimensions: 512 x 512 x 512 

==PROF== Connected to process 1922 (/content/laplace3d)
GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 120.7 (ms) 

20x Gold_laplace3d: 40849.2 (ms) 

==PROF== Profiling "GPU_laplace3d" - 0: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 1: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 2: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 3: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 4: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 5: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 6: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 7: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 8: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 9: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 10: 0%....50%....100% - 1 pass
==PROF== Profiling "GPU_laplace3d" - 11: 0%....50%..

In [ ]:
!ncu ./laplace3d
!ncu ./laplace3d_new

Grid dimensions: 512 x 512 x 512 

==PROF== Connected to process 2460 (/content/laplace3d)
GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 120.3 (ms) 

20x Gold_laplace3d: 41574.9 (ms) 

==PROF== Profiling "GPU_laplace3d" - 0: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 1: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 2: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 3: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 4: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 5: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 6: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 7: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 8: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 9: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplace3d" - 10: 0%....50%....100% - 9 passes
==PROF== Profiling "GPU_laplac

In [ ]:
from google.colab import runtime
runtime.unassign()